# Simple replacement for Kaldi's align-text

> "Because life's too short to install Kaldi again"

- toc: false
- badges: true
- categories: [align, difflib]

In [1]:
from difflib import SequenceMatcher
import copy

In [2]:
a = ['a', 'bad', 'time', 'today', 'etc']
b = ['not', 'really', 'bad', 'time', 'now']

In [3]:
s = SequenceMatcher(None, a, b)

In [4]:
def pad_replacements(a_in, b_in):
    a = copy.deepcopy(a_in)
    b = copy.deepcopy(b_in)
    if len(a) > len(b):
        diff = len(a) - len(b)
        for i in range(0, diff+1):
            b.append("<eps>")
    elif len(b) > len(a):
        diff = len(b) - len(a)
        for i in range(0, diff+1):
            a.append("<eps>")
    return [x for x in zip(a, b)]    

In [7]:
assert pad_replacements(["a", "b"], ["a"]) == [('a', 'a'), ('b', '<eps>')]
assert pad_replacements(["a", "b", "c"], ["a"]) == [('a', 'a'), ('b', '<eps>'), ('c', '<eps>')]
assert pad_replacements(["a"], ["a", "b", "c"]) == [('a', 'a'), ('<eps>', 'b'), ('<eps>', 'c')]


In [8]:
outputs = []
for tag, i1, i2, j1, j2 in s.get_opcodes():
    if tag == "equal":
        for x in a[i1:i2]:
            outputs.append(f"{x} {x}")
    elif tag == "insert":
        for x in b[j1:j2]:
            outputs.append(f"<eps> {x}")
    elif tag == "delete":
        for x in a[i1:i2]:
            outputs.append(f"{x} <eps>")
    elif tag == "replace":
        for x, y in pad_replacements(a[i1:i2], b[j1:j2]):
            outputs.append(f"{x} {y}")

In [9]:
" ; ".join(outputs)

'a not ; <eps> really ; bad bad ; time time ; today now ; etc <eps>'